In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
import json
import re

In [3]:
import pandas as pd

In [4]:
from itertools import chain
from operator import itemgetter

In [5]:
texts = []

In [6]:
with open("./gold/gold.txt") as f:
    texts = f.readlines()

In [7]:
gold = None
with open("./gold/gold.json") as f:
    gold = [json.loads(s) for s in f.readlines()]

In [8]:
paragraphs = list(map(itemgetter("Paragraph"), gold))
paragraph_sentences = list(map(itemgetter("Sentences"), gold))

### Features 

Задача классификации на два класса. -1 - разделитель предложений, 1 - обратно.

- если объект ! или ?, то скорее всего это разделитель.

Фичи:
- Factor: тип знака пунктуации (punctuation_kind: 0 - ., 1 - !, 2 - ?)
- Factor: тип следующего знака пунктуации (next_punctuation_kind)
- Factor: тип предыдущего знака пунктуации (prev_punctuation_kind)
- Numeric: кол-во символов до предыдущего разделителя (dist_to_prev)
- Numeric: кол-во символов до следующего разделителя (dist_to_next)
- Numeric: кол-во слов у предыдущего слова (len_of_prev_word)
- Numeric: кол-во слов у следующего слова (len_of_next_word)
- Logic: индикатор прописной буквы у предыдущего слова(считаем цифры и любую другую пунктуацию как строчные) (is_prev_uppercase)
- Logic: индикатор прописной буквы у следующего слова(считаем цифры и любую другую пунктуацию как пропиные) (is_next_uppercase)


- 5 млн и 7 млн руб., а затем снова про
- ить и боком. Честно-честно! X-Trail просто бе

**Выборка состоит из [Номер текста, позиция в тексте, знак пунктуации, стоит ли в конце текста, метка(-1 - разделитель)]**

In [9]:
from gold.feature_extraction import extract_features_from_labeled_data

In [10]:
data = list(chain.from_iterable(extract_features_from_labeled_data(paragraphs, paragraph_sentences)))

### Pandas

In [11]:
df = pd.DataFrame(data)
df.head()

,_label,_pos,_text_num,dist_to_next,dist_to_prev,is_next_uppercase,is_prev_uppercase,len_of_next_word,len_of_prev_word,next_punctuation_kind,prev_punctuation_kind,punctuation_kind
0,-1,218,0,62,219,1,1,2,8,.,|,.
1,1,280,0,9,62,1,1,7,1,.,.,.
2,-1,289,0,104,9,1,1,6,7,.,.,.
3,-1,393,0,10,104,1,0,1,6,.,.,.
4,1,403,0,3,10,0,0,1,1,.,.,.


In [12]:
df.to_csv("./dataset2.csv", index=False)